In [1]:
import altair as alt
import polars as pl

In [2]:
positions = pl.read_csv("positions.csv")

In [3]:
positions = positions.with_column(
    (pl.col("timestamp")+"Z").str.strptime(pl.Datetime, fmt="%+", strict=False)
)